# --- Day 15 Beacon Exclusion Zone ---

https://adventofcode.com/2022/day/15

## Get Input Data

Skip this step...  
Going to solve while reading & parsing the input data.

## Part 1
---

In [1]:
import re

In [2]:
def calc_man_dist(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

In [3]:
def solve1(filename, y_line=None):

    y_set = set()

    with open(f'../inputs/{filename}.txt') as file:
        for line in file:
            coords = [int(x) for x in re.findall(r"-?\d+", line)]  # Doh! Forgot to include "-?" in my regex... that was a hard bug to find!
            sensor = (coords[0], coords[1])  # (x, y) for sensors
            beacon = (coords[2], coords[3])  # (x, y) for beacons
            man_dist = calc_man_dist(sensor, beacon)

            if (sensor[1] - man_dist <= y_line <= sensor[1] + man_dist):
                y_diff = abs(sensor[1] - y_line)
                x_diff = abs(man_dist - y_diff)

                # Add range of x values in "exclusion zone" to the y_set
                y_set.update(range(sensor[0]-x_diff, sensor[0]+x_diff+1))

                # Remove x values for beacons
                if beacon[1] == y_line:
                    y_set.remove(beacon[0])

    return len(y_set)

### Run on Test Data

In [4]:
solve1('test_sensors_beacons', y_line=10) == 26

True

### Run on Input Data

In [5]:
%%time
solve1('sensors_beacons', y_line=2_000_000)

Wall time: 1.41 s


5607466

## Part 2
---

In [6]:
from collections import namedtuple

In [7]:
def get_data(filename):
    """Get the data for Part 2.

    Parameters
    ----------
    filename : str
        Name of file with input data.

    Returns
    -------
    list
        List of sensor info -- location and Manhattan distance of range.
    """

    Sensor = namedtuple('Sensor', ['loc', 'man_dist'])
    sensors = []

    with open(f'../inputs/{filename}.txt') as file:
        for line in file:
            coords = [int(x) for x in re.findall(r"-?\d+", line)]
            sensor_loc = (coords[0], coords[1])  # (x, y) for sensors
            beacon_loc = (coords[2], coords[3])  # (x, y) for beacons
            man_dist = calc_man_dist(sensor_loc, beacon_loc)
            
            sensors.append(Sensor(sensor_loc, man_dist))

    return sensors

In [8]:
get_data('test_sensors_beacons')

[Sensor(loc=(2, 18), man_dist=7),
 Sensor(loc=(9, 16), man_dist=1),
 Sensor(loc=(13, 2), man_dist=3),
 Sensor(loc=(12, 14), man_dist=4),
 Sensor(loc=(10, 20), man_dist=4),
 Sensor(loc=(14, 17), man_dist=5),
 Sensor(loc=(8, 7), man_dist=9),
 Sensor(loc=(2, 0), man_dist=10),
 Sensor(loc=(0, 11), man_dist=3),
 Sensor(loc=(20, 14), man_dist=8),
 Sensor(loc=(17, 20), man_dist=6),
 Sensor(loc=(16, 7), man_dist=5),
 Sensor(loc=(14, 3), man_dist=1),
 Sensor(loc=(20, 1), man_dist=7)]

In [9]:
def get_x_range(sensor, y_line, _range):
    """Get range of x values in the sesnor's _range, given the y_line we're at.

    Parameters
    ----------
    sensor : Sensor (namedtuple)
        Contains info on sensor location and Manhattan distance
    y_line : int
        Coordinate of line along the y axis to search.
    _range : int
        Upper end of range of x and y dimensions to search. (Test data = 20, actuall puzzle is 4_000_000)

    Returns
    -------
    tuple
        Min and max of x range.
    """
    
    y_diff = abs(sensor.loc[1] - y_line)
    x_diff = abs(sensor.man_dist - y_diff)

    x_range = (max(0, sensor.loc[0] - x_diff), min(_range, sensor.loc[0] + x_diff))
    return x_range

In [10]:
def solve2(sensors, _range):
    """Solve Part 2

    Find the *only* coordinate in the (0, _range) range (for both x and y) that 
    *CANNOT* have a beacon.

    Parameters
    ----------
    sensors : list of Sensors (namedtuples)
    _range : int
        The upper end of the range over which to search for the missing beacon.
        (The lower end is 0)

    Returns
    -------
    int
        4_000_000 * the x coordinate + the y coordinate of the missing beacon.
    """

    for y_line in range(0, _range + 1):
        # list of ranges each sensor covers for each y_line
        x_ranges = []

        # Go over each sensor (there are 27 in the input data)
        # If the y_line falls within the sensor's range, add the range to the list of ranges
        for sensor in sensors:
            if (sensor.loc[1] - sensor.man_dist <= y_line <= sensor.loc[1] + sensor.man_dist):
                x_ranges.append(get_x_range(sensor, y_line, _range))

        # Sort the ranges (which will sort on the first element of the x_range tuple)
        x_ranges = sorted(x_ranges)
        # Create an index var because some ranges will be within the previous range
        x_index = 0

        for i in range(len(x_ranges)-1):
            # Update the x_index value
            x_index = max(x_index, x_ranges[i][1])
            # If there's a gap of 2 between the x_index and the min of the next range, then that's
            # the x coordinate of the missing beacon. The y coordinate is the y_line value.
            if x_index + 2 == x_ranges[i+1][0]:
                x = x_index + 1
                return x * 4_000_000 + y_line

### Run on Test Data

In [11]:
solve2(get_data('test_sensors_beacons'), 20) == 56000011

True

### Run on Input Data

In [12]:
%%time
solve2(get_data('sensors_beacons'), 4_000_000)

Wall time: 1min 4s


12543202766584